In [5]:
! pip install -q tensorflow numpy
! cp drive/My\ Drive/Colab\ Notebooks/1_test/*.csv ./
! cp drive/My\ Drive/Colab\ Notebooks/*.pkl ./
! cp drive/My\ Drive/Colab\ Notebooks/*.gz ./
! cp -r drive/My\ Drive/Colab\ Notebooks/models/ensemble_test ./

In [35]:
def ohe(map):
  one_hot_encode = []
  temp = df[['Visual']].values

  for i in temp:
    arr = list(np.zeros(len(map),dtype=int))
    for j in i[0].split():
      if j.strip(',') not in map:
        pass
      elif j.strip(',') != '0':
        arr[map[j.strip(',')]] = 1
    one_hot_encode.append(arr)
  return np.array(one_hot_encode)

In [ ]:
def ensemble_calc(a, d, r):
  final = []
  for i in range(len(a)):
    if a[i][0] + d[i][0] + r[i][0] >= 2:
      final.append(1)
    else:
      final.append(0)
  final = np.array(final)
  final = final.reshape(final.shape[0], 1)
  return final

In [6]:
from tensorflow import keras

alex = keras.models.load_model('ensemble_test/alexnet_ffwd')
dense = keras.models.load_model('ensemble_test/densenet_ffwd')
resnet = keras.models.load_model('ensemble_test/resnet_ffwd')

In [7]:
import pandas as pd

df = pd.read_csv('meme_test_final.csv')
embedding = pd.read_csv('1_test_embeddings.csv', header=None)

In [38]:
import numpy as np
import pickle

des_X, alex_X, res_X = pickle.load(open('densenest_test.pkl', 'rb')), pickle.load(open('alexnet_test.pkl', 'rb')), np.array([embedding[1][i].split() for i in range(400)]).astype(float)

In [39]:
import joblib

ssc = joblib.load('ssc.gz')
mms = joblib.load('mms.gz')

In [40]:
import pickle
from datetime import date

temp = mms.transform(np.array([(date(int(i[0].split('/')[2]), int(i[0].split('/')[1]), int(i[0].split('/')[0])) - date(2015, 1, 1)).days for i in df[['Date']].values.tolist()]).reshape(400, 1))
print(temp.shape)
des_X = np.hstack((des_X, temp))
alex_X = np.hstack((alex_X, temp))
res_X = np.hstack((res_X, temp))
print(des_X.shape)
print(alex_X.shape)
print(res_X.shape)

temp = ssc.transform(df[['Engagement']].values)
print(temp.shape)
des_X = np.hstack((des_X, temp))
alex_X = np.hstack((alex_X, temp))
res_X = np.hstack((res_X, temp))
print(des_X.shape)
print(alex_X.shape)
print(res_X.shape)

temp = ohe(pickle.load(open('map.pkl', 'rb')))
print(temp.shape)
des_X = np.hstack((des_X, temp))
alex_X = np.hstack((alex_X, temp))
res_X = np.hstack((res_X, temp))
print(des_X.shape)
print(alex_X.shape)
print(res_X.shape)

temp = pickle.load(open('cls_umberto_test.pkl', 'rb'))
print(temp.shape)
des_X = np.hstack((des_X, temp))
alex_X = np.hstack((alex_X, temp))
res_X = np.hstack((res_X, temp))
print(des_X.shape)
print(alex_X.shape)
print(res_X.shape)

(400, 1)
(400, 1001)
(400, 4097)
(400, 2049)
(400, 1)
(400, 1002)
(400, 4098)
(400, 2050)
(400, 71)
(400, 1073)
(400, 4169)
(400, 2121)
(400, 768)
(400, 1841)
(400, 4937)
(400, 2889)


In [41]:
### Cumulative Score

a = (alex(alex_X) > 0.5).numpy().astype("int32").tolist()
d = (dense(des_X) > 0.5).numpy().astype("int32").tolist()
r = (resnet(res_X) > 0.5).numpy().astype("int32").tolist()
final = ensemble_calc(a, d, r)

In [44]:
print(final.ravel().tolist())

[1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 